# Named Entity Recognition using Language Models

This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

## Installation
Let us first install the required libraries

In [ ]:
!pip install datasets transformers seqeval

## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [1]:
from datasets import load_dataset, load_metric, concatenate_datasets
datasets = load_dataset("Babelscape/wikineural")

Using custom data configuration Babelscape--wikineural-c8d61e4987fde127
Reusing dataset parquet (/home/diptesh/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)
100%|██████████| 27/27 [00:00<00:00, 1181.25it/s]


The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [2]:
datasets["train_en"][0]

{'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'lang': 'en'}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [3]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

If you want to train a monolingual model (NER on a single language), you have to use a single language for each split.

In [4]:
train_dataset = datasets["train_en"]
val_dataset = datasets["val_en"]
test_dataset = datasets["test_en"]

This is how you can concatenate multiple dataset splits to train a Multilingual NER model. However, let us focus on Monolingual NER for now.

In [5]:
# train_dataset = concatenate_datasets([datasets["train_de"], 
#                                       datasets["train_en"], 
#                                       datasets["train_es"], 
#                                       datasets["train_fr"],
#                                       datasets["train_it"],
#                                       datasets["train_nl"],
#                                       datasets["train_pl"],
#                                       datasets["train_pt"],
#                                       datasets["train_ru"]])

# val_dataset = concatenate_datasets([datasets["val_de"], 
#                                     datasets["val_en"], 
#                                     datasets["val_es"], 
#                                     datasets["val_fr"],
#                                     datasets["val_it"],
#                                     datasets["val_nl"],
#                                     datasets["val_pl"],
#                                     datasets["val_pt"],
#                                     datasets["val_ru"]])

# test_dataset = concatenate_datasets([datasets["test_de"], 
#                                     datasets["test_en"], 
#                                     datasets["test_es"], 
#                                     datasets["test_fr"],
#                                     datasets["test_it"],
#                                     datasets["test_nl"],
#                                     datasets["test_pl"],
#                                     datasets["test_pt"],
#                                     datasets["test_ru"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.
 
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [7]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can directly call this tokenizer on one sentence, or you can use the tokenize method to tokenize multiple sentences at once.

In [8]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [9]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [11]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

100%|██████████| 93/93 [00:06<00:00, 15.48ba/s]
Loading cached processed dataset at /home/diptesh/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-6d38bc12d9fb7808.arrow
Loading cached processed dataset at /home/diptesh/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-260af9cbaaa86d4c.arrow


Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [12]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [13]:
model_name = "bert-base-multilingual-cased"
args = TrainingArguments(
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [15]:
metric = load_metric("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100
 
The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [16]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/diptesh/workspace/IIITL_NLP_Lab/Lab08/wikineural-multilingual-ner is already a clone of https://huggingface.co/dipteshkanojia/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens, lang.
***** Running training *****
  Num examples = 92720
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2898
 17%|█▋        | 500/2898 [01:42<08:22,  4.78it/s]

{'loss': 0.1102, 'learning_rate': 1.6549344375431333e-05, 'epoch': 0.17}


 35%|███▍      | 1000/2898 [03:25<06:07,  5.16it/s]

{'loss': 0.0384, 'learning_rate': 1.3098688750862665e-05, 'epoch': 0.35}


 52%|█████▏    | 1500/2898 [05:09<05:20,  4.37it/s]

...

 86%|████████▋ | 2501/2898 [08:37<01:25,  4.63it/s]

{'loss': 0.0299, 'learning_rate': 2.74672187715666e-06, 'epoch': 0.86}


100%|█████████▉| 2897/2898 [09:59<00:00,  4.89it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2898/2898 [09:59<00:00,  4.83it/s]

{'train_runtime': 599.66, 'train_samples_per_second': 154.621, 'train_steps_per_second': 4.833, 'train_loss': 0.045926961839569284, 'epoch': 1.0}


TrainOutput(global_step=2898, training_loss=0.045926961839569284, metrics={'train_runtime': 599.66, 'train_samples_per_second': 154.621, 'train_steps_per_second': 4.833, 'train_loss': 0.045926961839569284, 'epoch': 1.0})

## Evaluation

The evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:

In [19]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens, lang.
***** Running Evaluation *****
  Num examples = 11597
  Batch size = 16
100%|██████████| 725/725 [00:25<00:00, 28.53it/s]


{'eval_loss': 0.026731276884675026,
 'eval_precision': 0.921583459404942,
 'eval_recall': 0.928843710292249,
 'eval_f1': 0.9251993418554614,
 'eval_accuracy': 0.9913009301716232,
 'eval_runtime': 27.9731,
 'eval_samples_per_second': 414.577,
 'eval_steps_per_second': 25.918,
 'epoch': 1.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [20]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens, lang.
***** Running Prediction *****
  Num examples = 11597
  Batch size = 16
100%|██████████| 725/725 [00:21<00:00, 35.18it/s]

{'LOC': {'precision': 0.9286901595744681,
  'recall': 0.9382031905961377,
  'f1': 0.9334224375574305,
  'number': 5955},
 'MISC': {'precision': 0.8689160364553035,
  'recall': 0.8853981426595534,
  'f1': 0.8770796633392053,
  'number': 5061},
 'ORG': {'precision': 0.929737609329446,
  'recall': 0.9246158306755581,
  'f1': 0.9271696467509812,
  'number': 3449},
 'PER': {'precision': 0.9600153051463555,
  'recall': 0.963147792706334,
  'f1': 0.9615789977963016,
  'number': 5210},
 'overall_precision': 0.921583459404942,
 'overall_recall': 0.928843710292249,
 'overall_f1': 0.9251993418554614,
 'overall_accuracy': 0.9913009301716232}

100%|██████████| 725/725 [00:37<00:00, 35.18it/s]

Now, can you try to evaluate the NER task performance on a multilingual dataset? with a different language model?
1. Think about what Language Model on HuggingFace can help you Fine-tune over a multilingual dataset. Hint: XLM-R
2. Go back to the commented out multilingual dataset concatetation notebook cell and try to include a few more languages in the dataset.
3. Try to fine-tune the model on this dataset.
4. Report the performance in a MARKDOWN CELL below, showing your work on multilingual model fine-tuning in the cells below. 
5. Submit this notebook to Google Classroom